In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Structured-Output-Chain")

LangSmith 추적을 시작합니다.
[프로젝트명]
Structured-Output-Chain


In [2]:
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

class Quiz(BaseModel):
    """4지선다형 퀴즈의 정보를 추출"""

    question: str = Field(..., description="퀴즈의 질문")
    level: str = Field(
        ..., description="퀴즈의 난이도. (쉬움, 보통, 어려움)"
    )
    options: List[str] = Field(..., description="퀴즈의 4개의 선택지")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You're a world-famous quizzer and generates quizzes in structured formats."),
        ("human", "TOPIC에 제시된 내용과 관련한 4지선다형 퀴즈를 출제해 주세요. 만약, 실제 출제된 기출문제가 있다면 비슷한 문제를 만들어 출제하세요."
         "단, 문제에 TOPIC에 대한 내용이나 정보는 포함하지 마세요. \nTOPIC:\n{topic}"),
        ("human", "Tip: Make sure to answer in the correct format")
    ]
)

# 구조화된 출력을 위한 모델 생성
llm_with_structured_output = llm.with_structured_output(Quiz)

# 퀴즈 생성 체인 생성
chain = prompt | llm_with_structured_output

C:\Users\kakao\AI_study\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3549: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\kakao\AI_study\.venv\Lib\site-packages\langchain_openai\chat_models\base.py:1534: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="funct

In [3]:
# 퀴즈 생성을 요청
generated_quiz = chain.invoke({"topic": "ADSP(데이터 분석 준전문가) 자격 시험"})

In [4]:
# 생성된 퀴즈 출력
print(f"{generated_quiz.question} (난이도: {generated_quiz.level})\n")
for i, opt in enumerate(generated_quiz.options):
    print(f"{i+1}. {opt}")

ADSP 자격 시험에서 데이터 분석의 주요 단계 중 하나로, 데이터를 수집하고 정리하는 과정을 무엇이라고 하는가? (난이도: 보통)

1. 데이터 수집
2. 데이터 전처리
3. 데이터 분석
4. 데이터 시각화
